In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
#!pip install scikit-learn==0.24.0

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LogisticRegression

from sklearn.cluster import k_means
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
!pip install hdbscan
!pip install fa_kit
!pip install kneed
from fa_kit.rotation import VarimaxRotatorPython

from sklearn import metrics
import os

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model

import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

from scipy.spatial import distance
from scipy.cluster import hierarchy

import seaborn as sns; sns.set();
from seaborn import heatmap

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model

import random as random
from sklearn.multiclass import OneVsRestClassifier

from xgboost import XGBClassifier

import itertools as it
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import accuracy_score

from sklearn.neighbors import LocalOutlierFactor

In [16]:
#read in original data (wo anonymization)
def read_file_o(dfs, device):
    df = pd.DataFrame()
    for i in dfs:
        #df_new = pd.read_csv(i+'.csv')
        #df_new = pd.read_csv('./Simple_home_1003_sec_cam/'+i+'.csv')
        df_new = pd.read_csv('./'+device+'/'+i+'.csv')
        df_new['label'] = i
        df = pd.concat([df,df_new], ignore_index=True)
    return df

In [17]:
def read_file(folder, samp = False, lof=False):
    df = pd.DataFrame()
    path = folder+'/'
    df_new = pd.read_csv(path+'benign_traffic.csv')
    df_new['label'] = 'benign_traffic'
    if samp==True:
        df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
    if lof==True:
        #df_new = pd.DataFrame(scaler.fit_transform(df_new))
        df_new = filter_lof(df_new)
        #df_new = scaler.inverse_transform(df_new)
    df = pd.concat([df,df_new], ignore_index=True)
    
    directory = os.fsencode(path+'gafgyt_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'gafgyt_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'gafgyt_'+filename
            if samp==True:
                df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
            if lof==True:
                #df_new = pd.DataFrame(scaler.fit_transform(df_new))
                df_new = filter_lof(df_new)
                #df_new = scaler.inverse_transform(df_new)
            df = pd.concat([df,df_new], ignore_index=True)
    directory = os.fsencode(path+'mirai_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'mirai_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'mirai_'+filename
            if samp==True:
                df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
            if lof==True:
                #df_new = pd.DataFrame(scaler.fit_transform(df_new))
                df_new = filter_lof(df_new)
                #df_new = scaler.inverse_transform(df_new)
            df = pd.concat([df,df_new], ignore_index=True)
    return df

In [18]:
def pca_func(rate, data):    
    df_wo_label = data.drop(columns = ['label'])
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(df_wo_label)
    pca = PCA(n_components=rate, random_state=0)
    pca_comp = pca.fit(data_scaled).components_
    return pca_comp, data_scaled

In [19]:
# LEts transform our original dataset to lower dimensional
def decrease_dim(components, data_scaled):
    varimax = VarimaxRotatorPython()
    rotated_weights = varimax.rotate(components.T)
    df_lowdim = pd.DataFrame(np.dot(data_scaled, rotated_weights))
    #df_lowdim["label"] = df["label"]
    rotated_data = np.dot(data_scaled, rotated_weights)
    return df_lowdim, rotated_data

In [20]:
import scipy.special

def adj_rand(data):
    tab = pd.crosstab(data['label'], data['cluster_label'])
    df_tab = pd.DataFrame(tab)
    a = list(df_tab.sum(axis=1))
    b = list(df_tab.sum(axis=0))
    n = df_tab.values.tolist()
    sum_n = 0
    for i in range(len(n)):
        for j in range(len(n[0])):
            sum_n += scipy.special.comb(n[i][j],2)
    sum_a = 0
    for k in range(len(a)):
        sum_a += scipy.special.comb(a[k],2)
    sum_b = 0
    for l in range(len(b)):
        sum_b += scipy.special.comb(b[l],2)
    
    ari = (sum_n - sum_a*sum_b/scipy.special.comb(sum(a),2))/(0.5*(sum_a+sum_b)-sum_a*sum_b/scipy.special.comb(sum(a),2))
    return(ari)

In [21]:
def evall(df_cl):
    sc = []
    sc.append(metrics.rand_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(adj_rand(df_cl))
    sc.append(metrics.mutual_info_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.adjusted_mutual_info_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.homogeneity_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.completeness_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.v_measure_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.fowlkes_mallows_score(df_cl['label'], df_cl['cluster_label']))
    return sc

In [22]:
def kmeans(low_data, num_cl):
    low_data_2 = low_data.copy()
    kmeans = KMeans(init="random",n_clusters=num_cl,n_init=10,max_iter=200,random_state=43)
    kmeans.fit(low_data_2)
    low_data_2["cluster_label"] = kmeans.labels_
    low_data_2["label"] = df["label"]
    return low_data_2

In [23]:
def minibatch(low_data, num_cl):
    low_data_2 = low_data.copy()
    mini_kmeans = MiniBatchKMeans(init="random",n_clusters=num_cl,n_init=10,max_iter=200,random_state=43)
    mini_kmeans.fit(low_data_2)
    low_data_2["cluster_label"] = mini_kmeans.labels_
    low_data_2["label"] = df["label"]
    return low_data_2

In [24]:
def mean_shift(low_data):
    low_data_2 = low_data.copy()
    mean_sh = MeanShift().fit(low_data_2)
    low_data_2["cluster_label"] = mean_sh.labels_
    low_data_2["label"] = reduced_label
    return low_data_2

In [25]:
def optics(low_data):
    low_data_2 = low_data.copy()
    opti = OPTICS().fit(low_data_2)
    low_data_2["cluster_label"] = opti.labels_
    low_data_2["label"] = reduced_label
    return low_data_2

In [26]:
def dbscan(low_data, ep, min_samp):
    low_data_2 = low_data.copy()
    dbsc = DBSCAN(eps=ep, min_samples=min_samp).fit(low_data_2)
    low_data_2["cluster_label"] = dbsc.labels_
    low_data_2["label"] = reduced_label
    return low_data_2

In [27]:
import scipy.special

def adj_rand_2(label_pred, label_true):
    tab = pd.crosstab(label_true, label_pred)
    df_tab = pd.DataFrame(tab)
    a = list(df_tab.sum(axis=1))
    b = list(df_tab.sum(axis=0))
    n = df_tab.values.tolist()
    sum_n = 0
    for i in range(len(n)):
        for j in range(len(n[0])):
            sum_n += scipy.special.comb(n[i][j],2)
    sum_a = 0
    for k in range(len(a)):
        sum_a += scipy.special.comb(a[k],2)
    sum_b = 0
    for l in range(len(b)):
        sum_b += scipy.special.comb(b[l],2)
    
    ari = (sum_n - sum_a*sum_b/scipy.special.comb(sum(a),2))/(0.5*(sum_a+sum_b)-sum_a*sum_b/scipy.special.comb(sum(a),2))
    return(ari)

In [28]:
def evall_2(label_pred, label_true):
    sc = []
    sc.append(metrics.rand_score(label_true, label_pred))
    sc.append(adj_rand_2(label_true, label_pred))
    sc.append(metrics.mutual_info_score(label_true, label_pred))
    sc.append(metrics.adjusted_mutual_info_score(label_true, label_pred))
    sc.append(metrics.homogeneity_score(label_true, label_pred))
    sc.append(metrics.completeness_score(label_true, label_pred))
    sc.append(metrics.v_measure_score(label_true, label_pred))
    sc.append(metrics.fowlkes_mallows_score(label_true, label_pred))
    return sc

In [33]:
df_eval = pd.DataFrame(columns = ['rand_score', 'adjusted_rand_score', 'mutual_info_score', 'adjusted_mutual_info_score', 
                                  'homogeneity_score', 'completeness_score', 'v_measure_score', 'fowlkes_mallows_score'])
#for device in ['SimpleHome_XCS7_1003_WHT_Security_Camera', 'Philips_B120N10_Baby_Monitor']:
for device in ['SimpleHome_XCS7_1003_WHT_Security_Camera','Philips_B120N10_Baby_Monitor']:
    print(device)
    #original file beolvasas
    files = ['benign_traffic', 'bashlite_combo', 'bashlite_junk', 'bashlite_scan', 'bashlite_tcp', 'bashlite_udp', 'mirai_ack', 'mirai_scan', 'mirai_syn', 'mirai_udp', 'mirai_udpplain']
    #scaling
    df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True)
    df_reduced = df_original.sample(n=100000,replace=False)
    reduced_label = df_reduced['label'].reset_index(drop=True)
    comp, df_scaled = pca_func(0.7, df_reduced)
    df_low, np_low = decrease_dim(comp, df_scaled)
    ultim_best_scores = [0,0,0,0,0,0,0,0]
    df_ultim_best = pd.DataFrame()                        
    for algo in ['optics', 'kmeans', 'minibatch', 'dbscan']:
        print(algo)
        if algo is 'meanshift':
            df_meanshift = mean_shift(df_low)
            metrics_meanshift = evall(df_meanshift)
            if metrics_meanshift[6]>ultim_best_scores[6]:
                ultim_best_scores = metrics_meanshift
                df_ultim_best = df_meanshift
        if algo is 'optics':
            df_optics = optics(df_low)
            metrics_optics = evall(df_optics)
            if metrics_optics[6]>ultim_best_scores[6]:
                ultim_best_scores = metrics_optics
                df_ultim_best = df_optics
        if algo is 'kmeans':
            eval_kmeans = [0,0,0,0,0,0,0,0]
            df_best = pd.DataFrame()
            for k in range(7,15):
                df_kmeans = kmeans(df_low, k)
                metrics_kmeans = evall(df_kmeans)
                if metrics_kmeans[6]>eval_kmeans[6]:
                    eval_kmeans=metrics_kmeans
                    df_best_kmeans = df_kmeans
            if eval_kmeans[6]>ultim_best_scores[6]:
                ultim_best_scores = eval_kmeans
                df_ultim_best = df_best_kmeans
        if algo is 'minibatch':
            eval_minibatch = [0,0,0,0,0,0,0,0]
            df_best = pd.DataFrame()
            for k in range(7,15):
                df_minibatch = minibatch(df_low, k)
                metrics_minibatch = evall(df_minibatch)
                if metrics_minibatch[6]>eval_minibatch[6]:
                    eval_minibatch=metrics_minibatch
                    df_best_minibatch = df_minibatch
                if eval_minibatch[6]>ultim_best_scores[6]:
                    ultim_best_scores = eval_minibatch
                    df_ultim_best = df_best_minibatch
        if algo is 'dbscan':
            eval_dbscan = [0,0,0,0,0,0,0,0]
            df_best = pd.DataFrame()
            for ep in [0.18,0.2,0.22,0.25,0.3,0.4,0.5]:
                for min_sam in [3,5,8,10,12]:
                    df_dbscan = dbscan(df_low, ep, min_sam)
                    metrics_dbscan = evall(df_dbscan)
                    if metrics_dbscan[6]>eval_dbscan[6]:
                        eval_dbscan=metrics_dbscan
                        df_best_dbscan = df_dbscan
            if eval_dbscan[6]>ultim_best_scores[6]:
                ultim_best_scores = eval_dbscan
                df_ultim_best = df_best_dbscan
    print(ultim_best_scores)

SimpleHome_XCS7_1003_WHT_Security_Camera
optics
kmeans
minibatch
dbscan
[0.8752314257142572, 0.4787536570722892, 1.523188557253452, 0.6927589680901312, 0.6817624784094604, 0.7056477879870281, 0.693499531428013, 0.5551218598933739]
Philips_B120N10_Baby_Monitor
optics
kmeans
minibatch
dbscan
[0.9154562245622456, 0.6018793261670413, 1.7763790787281346, 0.7857594010976108, 0.7896472173794143, 0.7822660136029327, 0.7859392856004022, 0.6500980071361933]
